# Line direction inference

In [5]:
import geopandas as gpd
from tqdm import tqdm
import pandas as pd
import folium
import json

In [6]:
lines = gpd.read_file("data/map/2109_STIB_MIVB_Network/ACTU_LINES.shp")
stops_geom = gpd.read_file("data/map/2109_STIB_MIVB_Network/ACTU_STOPS.shp")

In [75]:
lines

,LIGNE,VARIANTE,COLOR_HEX,Date_debut,Date_fin,geometry
0,001m,1,#C4008F,01/09/2021,06/03/2022,"LINESTRING Z (146633.500 170956.400 0.000, 146..."
1,001m,2,#C4008F,01/09/2021,06/03/2022,"LINESTRING Z (156746.700 170167.000 0.000, 156..."
2,002m,1,#F57000,01/09/2021,06/03/2022,"LINESTRING Z (147305.500 172526.900 0.000, 147..."
3,002m,2,#F57000,01/09/2021,06/03/2022,"LINESTRING Z (147370.500 172498.500 0.000, 147..."
4,003t,1,#B5BA05,01/09/2021,06/03/2022,"LINESTRING Z (148550.000 176641.300 0.000, 148..."
...,...,...,...,...,...,...
169,213b,2,#991F36,01/09/2021,06/03/2022,"LINESTRING Z (144054.200 169925.500 0.000, 144..."
170,216b,1,#80C29C,01/09/2021,06/03/2022,"LINESTRING Z (149229.000 170664.500 0.000, 149..."
171,216b,2,#80C29C,01/09/2021,06/03/2022,"LINESTRING Z (144443.400 173553.200 0.000, 144..."
172,218b,1,#17A345,01/09/2021,06/03/2022,"LINESTRING Z (149173.500 170683.400 0.000, 149..."


In [76]:
stops_geom

,Code_Ligne,Variante,succession,stop_id,descr_fr,descr_nl,alpha_fr,alpha_nl,coord_x,coord_y,mode,numero_lig,terminus,geometry
0,012b,1,1,9600B,BRUSSELS AIRPORT,BRUSSELS AIRPORT,Brussels Airport,Brussels Airport,157950.0,176429.0,B,12,BRUSSELS CITY,POINT (157950.000 176429.000)
1,012b,1,2,3017,BOURGET,BOURGET,Bourget,Bourget,154334.0,174200.0,B,12,BRUSSELS CITY,POINT (154334.000 174200.000)
2,012b,1,3,5048,DA VINCI,DA VINCI,Da Vinci,Da Vinci,152934.0,173976.0,B,12,BRUSSELS CITY,POINT (152934.000 173976.000)
3,012b,1,4,2695,GENEVE,GENEVE,Genève,Genève,152428.0,172606.0,B,12,BRUSSELS CITY,POINT (152428.000 172606.000)
4,012b,1,5,2250,MEISER,MEISER,Meiser,Meiser,152045.0,171508.0,B,12,BRUSSELS CITY,POINT (152045.000 171508.000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4171,097t,2,25,6427F,JANSON,JANSON,Janson,Janson,148985.0,168336.0,T,97,LOUISE,POINT (148985.000 168336.000)
4172,097t,2,26,6430F,JANSON,JANSON,Janson,Janson,149019.0,168413.0,T,97,LOUISE,POINT (149019.000 168413.000)
4173,097t,2,27,5066F,FAIDER,FAIDER,Faider,Faider,149107.0,168847.0,T,97,LOUISE,POINT (149107.000 168847.000)
4174,097t,2,28,5068F,STEPHANIE,STEFANIA,Stéphanie,Stefania,149184.6,169237.5,T,97,LOUISE,POINT (149184.600 169237.500)


In [ ]:
map = folium.Map(location= [50.85045, 4.34878],  zoom_start=12, tiles="cartodbpositron",)

m = lines.explore( 
    column = "LIGNE",
    cmap="Set1",
    m = map
)
stops_geom.explore(
    m=m,
    color="red")

In [77]:
success_stops = stops_geom.groupby(by = ["Code_Ligne", "Variante"])

In [40]:
d = {}
for name, group in success_stops:
    temp = d.get(group.iloc[0].Code_Ligne, {})
    temp[int(group.iloc[0].Variante)] = group.sort_values(by=["succession"]).stop_id.to_list()
    d[group.iloc[0].Code_Ligne] = temp
    
with open("data/computed/route_success.json", "w") as f:
    json.dump(d, f)

In [78]:
i = 0
for name, group in success_stops:
    if i == 0:
        sample = group
    

In [84]:
sample = success_stops.get_group(("007t", 2))
sample

,Code_Ligne,Variante,succession,stop_id,descr_fr,descr_nl,alpha_fr,alpha_nl,coord_x,coord_y,mode,numero_lig,terminus,geometry
3305,007t,2,1,9056F,HEYSEL,HEIZEL,Heysel,Heizel,147847.8,176230.4,T,7,VANDERKINDERE,POINT (147847.800 176230.400)
3306,007t,2,2,2636F,SAINT-LAMBERT,SINT-LAMBERTUS,Saint-Lambert,Sint-Lambertus,148403.0,175613.0,T,7,VANDERKINDERE,POINT (148403.000 175613.000)
3307,007t,2,3,5700G,DE WAND,DE WAND,De Wand,De Wand,149205.7,176083.5,T,7,VANDERKINDERE,POINT (149205.700 176083.500)
3308,007t,2,4,5704F,ARAUCARIA,ARAUCARIA,Araucaria,Araucaria,149662.5,175835.7,T,7,VANDERKINDERE,POINT (149662.500 175835.700)
3309,007t,2,5,5705,BUISSONNETS,BRAAMBOSJES,Buissonnets,Braambosjes,150008.0,175660.0,T,7,VANDERKINDERE,POINT (150008.000 175660.000)
3310,007t,2,6,5706,HEEMBEEK,HEEMBEEK,Heembeek,Heembeek,150457.0,175346.0,T,7,VANDERKINDERE,POINT (150457.000 175346.000)
3311,007t,2,7,5707,VAN PRAET,VAN PRAET,Van Praet,Van Praet,150320.0,174580.0,T,7,VANDERKINDERE,POINT (150320.000 174580.000)
3312,007t,2,8,5758F,DOCKS BRUXSEL,DOCKS BRUXSEL,Docks Bruxsel,Docks Bruxsel,150428.0,174312.0,T,7,VANDERKINDERE,POINT (150428.000 174312.000)
3313,007t,2,9,6457F,PRINC. ELISABETH,PRINS. ELISABETH,Princesse Élisabeth,Prinses Elisabeth,150659.0,173625.0,T,7,VANDERKINDERE,POINT (150659.000 173625.000)
3314,007t,2,10,5303,DEMOLDER,DEMOLDER,Demolder,Demolder,150830.0,173492.0,T,7,VANDERKINDERE,POINT (150830.000 173492.000)


In [85]:
sample['geom_position_before'] = sample.geometry.shift(1)
sample

/Users/mjdaoudi/opt/anaconda3/lib/python3.9/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as BD72 / Belgian Lambert 72 (the single non-null crs provided).
  warnings.warn(
/Users/mjdaoudi/opt/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,Code_Ligne,Variante,succession,stop_id,descr_fr,descr_nl,alpha_fr,alpha_nl,coord_x,coord_y,mode,numero_lig,terminus,geometry,geom_position_before
3305,007t,2,1,9056F,HEYSEL,HEIZEL,Heysel,Heizel,147847.8,176230.4,T,7,VANDERKINDERE,POINT (147847.800 176230.400),None
3306,007t,2,2,2636F,SAINT-LAMBERT,SINT-LAMBERTUS,Saint-Lambert,Sint-Lambertus,148403.0,175613.0,T,7,VANDERKINDERE,POINT (148403.000 175613.000),POINT (147847.800 176230.400)
3307,007t,2,3,5700G,DE WAND,DE WAND,De Wand,De Wand,149205.7,176083.5,T,7,VANDERKINDERE,POINT (149205.700 176083.500),POINT (148403.000 175613.000)
3308,007t,2,4,5704F,ARAUCARIA,ARAUCARIA,Araucaria,Araucaria,149662.5,175835.7,T,7,VANDERKINDERE,POINT (149662.500 175835.700),POINT (149205.700 176083.500)
3309,007t,2,5,5705,BUISSONNETS,BRAAMBOSJES,Buissonnets,Braambosjes,150008.0,175660.0,T,7,VANDERKINDERE,POINT (150008.000 175660.000),POINT (149662.500 175835.700)
3310,007t,2,6,5706,HEEMBEEK,HEEMBEEK,Heembeek,Heembeek,150457.0,175346.0,T,7,VANDERKINDERE,POINT (150457.000 175346.000),POINT (150008.000 175660.000)
3311,007t,2,7,5707,VAN PRAET,VAN PRAET,Van Praet,Van Praet,150320.0,174580.0,T,7,VANDERKINDERE,POINT (150320.000 174580.000),POINT (150457.000 175346.000)
3312,007t,2,8,5758F,DOCKS BRUXSEL,DOCKS BRUXSEL,Docks Bruxsel,Docks Bruxsel,150428.0,174312.0,T,7,VANDERKINDERE,POINT (150428.000 174312.000),POINT (150320.000 174580.000)
3313,007t,2,9,6457F,PRINC. ELISABETH,PRINS. ELISABETH,Princesse Élisabeth,Prinses Elisabeth,150659.0,173625.0,T,7,VANDERKINDERE,POINT (150659.000 173625.000),POINT (150428.000 174312.000)
3314,007t,2,10,5303,DEMOLDER,DEMOLDER,Demolder,Demolder,150830.0,173492.0,T,7,VANDERKINDERE,POINT (150830.000 173492.000),POINT (150659.000 173625.000)


In [86]:
sample

,Code_Ligne,Variante,succession,stop_id,descr_fr,descr_nl,alpha_fr,alpha_nl,coord_x,coord_y,mode,numero_lig,terminus,geometry,geom_position_before
3305,007t,2,1,9056F,HEYSEL,HEIZEL,Heysel,Heizel,147847.8,176230.4,T,7,VANDERKINDERE,POINT (147847.800 176230.400),None
3306,007t,2,2,2636F,SAINT-LAMBERT,SINT-LAMBERTUS,Saint-Lambert,Sint-Lambertus,148403.0,175613.0,T,7,VANDERKINDERE,POINT (148403.000 175613.000),POINT (147847.800 176230.400)
3307,007t,2,3,5700G,DE WAND,DE WAND,De Wand,De Wand,149205.7,176083.5,T,7,VANDERKINDERE,POINT (149205.700 176083.500),POINT (148403.000 175613.000)
3308,007t,2,4,5704F,ARAUCARIA,ARAUCARIA,Araucaria,Araucaria,149662.5,175835.7,T,7,VANDERKINDERE,POINT (149662.500 175835.700),POINT (149205.700 176083.500)
3309,007t,2,5,5705,BUISSONNETS,BRAAMBOSJES,Buissonnets,Braambosjes,150008.0,175660.0,T,7,VANDERKINDERE,POINT (150008.000 175660.000),POINT (149662.500 175835.700)
3310,007t,2,6,5706,HEEMBEEK,HEEMBEEK,Heembeek,Heembeek,150457.0,175346.0,T,7,VANDERKINDERE,POINT (150457.000 175346.000),POINT (150008.000 175660.000)
3311,007t,2,7,5707,VAN PRAET,VAN PRAET,Van Praet,Van Praet,150320.0,174580.0,T,7,VANDERKINDERE,POINT (150320.000 174580.000),POINT (150457.000 175346.000)
3312,007t,2,8,5758F,DOCKS BRUXSEL,DOCKS BRUXSEL,Docks Bruxsel,Docks Bruxsel,150428.0,174312.0,T,7,VANDERKINDERE,POINT (150428.000 174312.000),POINT (150320.000 174580.000)
3313,007t,2,9,6457F,PRINC. ELISABETH,PRINS. ELISABETH,Princesse Élisabeth,Prinses Elisabeth,150659.0,173625.0,T,7,VANDERKINDERE,POINT (150659.000 173625.000),POINT (150428.000 174312.000)
3314,007t,2,10,5303,DEMOLDER,DEMOLDER,Demolder,Demolder,150830.0,173492.0,T,7,VANDERKINDERE,POINT (150830.000 173492.000),POINT (150659.000 173625.000)


In [87]:
sample["dist_m"] = sample['geometry'].distance(sample["geom_position_before"])

sample.merge(
    lines,
    "left",
    left_on = ["Code_Ligne", "Variante"],
    right_on = ["LIGNE", "VARIANTE"]
)

/Users/mjdaoudi/opt/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,Code_Ligne,Variante,succession,stop_id,descr_fr,descr_nl,alpha_fr,alpha_nl,coord_x,coord_y,...,terminus,geometry_x,geom_position_before,dist_m,LIGNE,VARIANTE,COLOR_HEX,Date_debut,Date_fin,geometry_y
0,007t,2,1,9056F,HEYSEL,HEIZEL,Heysel,Heizel,147847.8,176230.4,...,VANDERKINDERE,POINT (147847.800 176230.400),None,NaN,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."
1,007t,2,2,2636F,SAINT-LAMBERT,SINT-LAMBERTUS,Saint-Lambert,Sint-Lambertus,148403.0,175613.0,...,VANDERKINDERE,POINT (148403.000 175613.000),POINT (147847.800 176230.400),830.319095,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."
2,007t,2,3,5700G,DE WAND,DE WAND,De Wand,De Wand,149205.7,176083.5,...,VANDERKINDERE,POINT (149205.700 176083.500),POINT (148403.000 175613.000),930.428686,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."
3,007t,2,4,5704F,ARAUCARIA,ARAUCARIA,Araucaria,Araucaria,149662.5,175835.7,...,VANDERKINDERE,POINT (149662.500 175835.700),POINT (149205.700 176083.500),519.683635,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."
4,007t,2,5,5705,BUISSONNETS,BRAAMBOSJES,Buissonnets,Braambosjes,150008.0,175660.0,...,VANDERKINDERE,POINT (150008.000 175660.000),POINT (149662.500 175835.700),387.609004,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."
5,007t,2,6,5706,HEEMBEEK,HEEMBEEK,Heembeek,Heembeek,150457.0,175346.0,...,VANDERKINDERE,POINT (150457.000 175346.000),POINT (150008.000 175660.000),547.902364,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."
6,007t,2,7,5707,VAN PRAET,VAN PRAET,Van Praet,Van Praet,150320.0,174580.0,...,VANDERKINDERE,POINT (150320.000 174580.000),POINT (150457.000 175346.000),778.154869,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."
7,007t,2,8,5758F,DOCKS BRUXSEL,DOCKS BRUXSEL,Docks Bruxsel,Docks Bruxsel,150428.0,174312.0,...,VANDERKINDERE,POINT (150428.000 174312.000),POINT (150320.000 174580.000),288.942901,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."
8,007t,2,9,6457F,PRINC. ELISABETH,PRINS. ELISABETH,Princesse Élisabeth,Prinses Elisabeth,150659.0,173625.0,...,VANDERKINDERE,POINT (150659.000 173625.000),POINT (150428.000 174312.000),724.796523,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."
9,007t,2,10,5303,DEMOLDER,DEMOLDER,Demolder,Demolder,150830.0,173492.0,...,VANDERKINDERE,POINT (150830.000 173492.000),POINT (150659.000 173625.000),216.633331,007t,2,#ECE354,01/09/2021,06/03/2022,"LINESTRING Z (147846.100 176232.100 0.000, 147..."


In [88]:
map = folium.Map(location= [50.85045, 4.34878],  zoom_start=12, tiles="cartodbpositron",)

m = sample.explore( 
    column = "stop_id",
    m = map,
    highlight = True
)
lines[(lines.LIGNE == sample.iloc[0].Code_Ligne) & (lines.VARIANTE == sample.iloc[0].Variante)].explore(
    m=m,
    color="red")
m